# Sesión 4.1 - Predecir la opinión de una reseña de una película
Curso 2022-23

Profesor: [Jorge Calvo Zaragoza](mailto:jcalvo@dlsi.ua.es)

## Resumen
En esta sesión:
  * Practicaremos el uso de redes recurrentes.
  * Resolveremos el problema de predecir si una opinión sobre una película es positiva o negativa.

## Formulación del problema

Vamos a plantear el problema de clasificar reseñas de películas utilizando el texto de la propia reseña. Será una formulación de clasificación binaria (de dos clases), ya que el modelo aprenderá simplemente a decir si la reseña es positiva (1) o negativa (0).

Usaremos el conjunto de datos **IMDB** que contiene el texto de 50.000 reseñas de películas y que viene incluida en el API de Keras.

### Carga de datos


El siguiente código realiza la llamada al API de Keras que descarga los datos de imdb. Esta llamada tiene una serie de parámetros para controlar las características de los datos. Más información en [la web de Keras](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification).

In [1]:
import numpy as np
import tensorflow as tf

# ----------------------------------------------------
# Cargamos los datos de la base de datos imdb
# - Podemos seleccionar algunas características como

LONGITUD_MAXIMA = 200 # Longitud máx. de la reseña (afecta al no. de muestras)
NUM_PALABRAS = 2000  # No. de palabras diferentes (no afecta al no. de muestras)

print('Cargando datos...')

# Los datos vienen divididos en entrenamiento y test
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(maxlen=LONGITUD_MAXIMA,
                                num_words=NUM_PALABRAS)

# Inspeccionemos los datos
print()
print('Formato datos: X {} - Y {}'.format( x_train.shape, y_train.shape ))
print()
print('Ejemplo secuencia de entrada: {}'.format(x_train[0]))
print('Ejemplo etiqueta: {}'.format(y_train[0]))

print()
print('Entrenamiento: X {} - Y {}'.format( x_train.shape, y_train.shape ))
print('Test: X {} - Y {}'.format( x_test.shape, y_test.shape ))



Cargando datos...
17464789/17464789 [==============================] - 1s 0us/step

Formato datos: X (14244,) - Y (14244,)

Ejemplo secuencia de entrada: [1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 2, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 2, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 2, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 2, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 2, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 2, 228, 2, 5, 2, 656, 245, 2, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]
Ejemplo etiqueta: 0

Entrenamiento: X (14244,) - Y (14244,)
Test: X (14

Podemos ver que las secuencias de las reseñas ya están pre-procesadas y aparecen codificadas como una secuencia de números enteros. Cada número, como hemos introducido en la sesión de teoría, identifica una única palabra del vocabulario.

Para continuar con el ejercicio, vamos a crear los diccionarios para traducir de números a palabras y viceversa.

In [6]:

# --------------------------------------------------------------------------
# Cargamos el diccionario que mapea palabras a un entero desde el API
word_to_int = tf.keras.datasets.imdb.get_word_index()

print()
print('Número de palabras usadas: {}'.format(NUM_PALABRAS))

# Tenemos que reservar las cuatro primeras posiciones para palabras especiales
word_to_int = {k:(v+3) for k,v in word_to_int.items()}
word_to_int["<PAD>"] = 0
word_to_int["<START>"] = 1
word_to_int["<UNK>"] = 2
word_to_int["<UNUSED>"] = 3

# Creamos el codificador reverso
int_to_word = dict([(value, key) for (key, value) in word_to_int.items()])

# Podemos crear un decodificador de la secuencia numerica a textual
def decodificar(secuencia):
    return ' '.join([int_to_word.get(palabra) for palabra in secuencia])

# Ejemplo decodificacion
print()
print('Ejemplo secuencia de entrada: {}'.format(decodificar(x_train[0])))
print('Ejemplo etiqueta: {}'.format(y_train[0]))
print()
print('Ejemplo secuencia de entrada: {}'.format(decodificar(x_train[3])))
print('Ejemplo etiqueta: {}'.format(y_train[3]))



Número de palabras usadas: 2000

Ejemplo secuencia de entrada: <START> big hair big <UNK> bad music and a giant <UNK> <UNK> these are the words to best describe this terrible movie i love cheesy horror movies and i've seen <UNK> but this had got to be on of the worst ever made the plot is <UNK> thin and ridiculous the acting is an <UNK> the script is completely laughable the best is the end <UNK> with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are <UNK> and funny in <UNK> <UNK> the hair is big lots of <UNK> <UNK> men <UNK> those cut <UNK> <UNK> that show off their <UNK> <UNK> that men actually <UNK> them and the music is just <UNK> trash that plays over and over again in almost every scene there is <UNK> music <UNK> and <UNK> taking away <UNK> and the <UNK> still doesn't close for <UNK> all <UNK> aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad ever

In [7]:
print('Ejemplo secuencia de entrada: {}'.format(decodificar(x_train[17])))
print('Ejemplo etiqueta: {}'.format(y_train[17]))

Ejemplo secuencia de entrada: <START> <UNK> <UNK> appeared in several of these low budget <UNK> for <UNK> <UNK> in the <UNK> and the <UNK> <UNK> is one of the better ones br br <UNK> plays a mad scientist who <UNK> young <UNK> and kills them and then <UNK> <UNK> from their <UNK> so he can keep his <UNK> wife looking young after a <UNK> and a doctor stay the night at his home and discover he is responsible for the <UNK> <UNK> the following morning they <UNK> these murders to the police and the mad scientist is shot and <UNK> dead <UNK> <UNK> br br you have got almost everything in this movie the <UNK> <UNK> <UNK> of an old <UNK> a <UNK> and <UNK> her <UNK> a <UNK> and <UNK> <UNK> in <UNK> house <UNK> and his wife find they sleep better in <UNK> rather than <UNK> in the movie br br the <UNK> <UNK> is worth a look especially for <UNK> <UNK> fans great fun br br rating 3 stars out of 5
Ejemplo etiqueta: 1


Siguiendo los principios introducidos en la sesión anterior, vamos a crear la función generadora que sirva los datos.

En este caso, los conjuntos X e Y se pasan por parámetro porque no se crean dinámicamente sino que están ya almacenados en variables.

In [45]:
# Crear una función generadora que acepte como parámetro:
# - X: secuencias que codifican una reseña
# - Y: etiqueta de reseña negativa (0) o positiva (1)
# - Tamaño de cada lote de datos

def generador(X, Y, tam_lote):
  idx = 0

  # Bucle infinito del generador
  while True:

    X_next =   X[idx:idx+tam_lote]   # TODO: Coge el siguiente lote de X (desde idx hasta idx+tam_lote)
    Y_next =   Y[idx:idx+tam_lote]   # TODO: Coge el siguiente lote de Y (desde idx hasta idx+tam_lote)

    max_longitud = max([len(b) for b in X])        # TODO Calcula la longitud de la secuencia más larga de X de este lote

    # Creamos el lote
    X_lote = np.zeros((tam_lote, max_longitud), dtype=np.int32)
    Y_lote = np.zeros((tam_lote, 1), dtype=np.int32)


    for idx_s, secuencia in enumerate(X_next):
      for idx_p, palabra in enumerate(secuencia):
          X_lote[idx_s][idx_p] = palabra

    for idx_e, etiqueta in enumerate(Y_next):
      Y_lote[idx_e] = etiqueta

    yield X_lote, Y_lote           # TODO Devolvemos el lote para el generador


    # Actualizamos `idx` para el próximo lote
    idx = (idx + tam_lote) % len(X)

# ----------------------------------------------------------------------
# Prueba de la función generadora

g = generador(x_train,y_train, tam_lote = 4)
X_lote, Y_lote = next(g)

print('Forma del lote X: {}'.format(X_lote.shape))
print('Forma del lote Y: {}'.format(Y_lote.shape))


Forma del lote X: (4, 199)
Forma del lote Y: (4, 1)


In [44]:
X_lote, Y_lote = next(g)

print('Forma del lote X: {}'.format(X_lote.shape))
print('Forma del lote Y: {}'.format(Y_lote.shape))

Forma del lote X: (4, 199)
Forma del lote Y: (4, 1)


Ahora podemos a construir el modelo de red. Al igual que en el ejemplo anterior, la entrada vendrá definida por secuencias de longitud variable y el número de características de cada elemento de la secuencia. Utilizaremos una capa *LSTM* y una *Dense* que realizará la clasificación.

In [53]:
%%capture --no-stdout

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Activation

# ----------------------------------------
# Construimos el modelo

print('Construyendo el modelo...')

# ===========
model =Sequential() # TODO: Define un modelo secuencial
model.add(Embedding(input_dim=NUM_PALABRAS, output_dim=128)) # TODO: Añade una capa embedding de 128 dimensiones
model.add(tf.keras.layers.LSTM(256)) # TODO: Añade una capa recurrente de 256 neuronas
model.add(Dense(1)) # TODO: Añade la capa densa correspondiente (¿qué activación tendrá?)
model.add(Activation('sigmoid'))

# ===========

model.compile(loss="mean_squared_error", # TODO: Define un loss apropiado para este modelo
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Construyendo el modelo...
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 128)         256000    
                                                                 
 lstm_6 (LSTM)               (None, 256)               394240    
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
 activation_6 (Activation)   (None, 1)                 0         
                                                                 
Total params: 650,497
Trainable params: 650,497
Non-trainable params: 0
_________________________________________________________________


Con el modelo construido y la función generadora definida, podemos realizar el entrenamiento. Para ello:
* Crearemos un generador para los datos de entrenamiento
* Crearemos un generador para los datos de test
* Llamaremos a *fit* con ambos generadores para entrenar y monitorizar.

In [54]:
%%capture --no-stdout

# ----------------------------------------
# Entrenamiento

TAM_LOTE = 32

print()
print('Entrenando la red...')

# Creamos las funciones generadores definitivas
gen_train = generador(x_train, y_train, TAM_LOTE)
gen_test = generador(x_test, y_test, TAM_LOTE)

# Calculamos el numero de pasos por epoca
pasos_por_epoca_train =  x_train.shape[0] // TAM_LOTE
pasos_por_epoca_val = x_test.shape[0] // TAM_LOTE

# TODO: Completa la llamada a fit
history = model.fit(gen_train,
                    steps_per_epoch=pasos_por_epoca_train,
                    validation_data=gen_test,
                    validation_steps=pasos_por_epoca_val,
                    epochs=15,
                    verbose=1)


Entrenando la red...
Epoch 1/15
445/445 [==============================] - 34s 71ms/step - loss: 0.2503 - accuracy: 0.4988 - val_loss: 0.2501 - val_accuracy: 0.4944
Epoch 2/15
445/445 [==============================] - 12s 26ms/step - loss: 0.2512 - accuracy: 0.5118 - val_loss: 0.2516 - val_accuracy: 0.5375
Epoch 3/15
445/445 [==============================] - 10s 23ms/step - loss: 0.2490 - accuracy: 0.5326 - val_loss: 0.2587 - val_accuracy: 0.4948
Epoch 4/15
445/445 [==============================] - 12s 27ms/step - loss: 0.2522 - accuracy: 0.4975 - val_loss: 0.2528 - val_accuracy: 0.4946
Epoch 5/15
445/445 [==============================] - 12s 27ms/step - loss: 0.2516 - accuracy: 0.4971 - val_loss: 0.2511 - val_accuracy: 0.4954
Epoch 6/15
445/445 [==============================] - 10s 22ms/step - loss: 0.2514 - accuracy: 0.5009 - val_loss: 0.2508 - val_accuracy: 0.4950
Epoch 7/15
445/445 [==============================] - 12s 27ms/step - loss: 0.2513 - accuracy: 0.5032 - val_loss: 

Parece ser que la red sí aprende a valorar si una reseña es positiva o negativa en base a las palabras utilizadas.

Vamos a inspeccionar detalladamente un lote de validación.

In [56]:
lote_validacion_x, lote_validacion_y = next(gen_test)

prediccion = model.predict(lote_validacion_x) # TODO: Predice sobre el lote de validacion

for idx_s, secuencia in enumerate(lote_validacion_x):
  print('Secuencia: {}'.format( decodificar(secuencia) ))
  print('Critica real: {}'.format( lote_validacion_y[idx_s] ))
  print('Prediccion: {}'.format( prediccion[idx_s] ))
  print()



1/1 [==============================] - 0s 19ms/step
Secuencia: <START> i began watching this movie on t v some <UNK> ago but gave up after the first 10 minutes or so at the start the person on the <UNK> <UNK> <UNK> <UNK> in an <UNK> <UNK> becomes <UNK> about his <UNK> <UNK> <UNK> and then asks the <UNK> inside the house whether he can make a <UNK> call only to discover the line has been cut br br <UNK> <UNK> <UNK> <UNK> as one of the <UNK> turns up and <UNK> <UNK> the <UNK> and his wife upon which the <UNK> <UNK> character <UNK> <UNK> up the wall <UNK> in the <UNK> a number and then <UNK> it is done that was it for me <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

Por último, vamos a realizar predicciones sobre secuencias creadas *manualmente* por nosotros.

In [57]:
import numpy as np


def predecir(secuencia):
  x = np.zeros((1, len(secuencia)), dtype=np.int32)

  for idx_p, palabra in enumerate(secuencia):
    x[0][idx_p] = word_to_int[palabra]

  prediccion = model.predict(x, verbose=0)[0]

  print('Crítica: {}'.format(secuencia))
  print('Predicción: {}'.format(prediccion))
  print()


predecir(['the', 'movie', 'is', 'the', 'best', 'ever'])
predecir(['the', 'movie', 'is', 'awful'])
predecir(['the', 'movie', 'is', '<UNK>'])

Crítica: ['the', 'movie', 'is', 'the', 'best', 'ever']
Predicción: [0.41724437]

Crítica: ['the', 'movie', 'is', 'awful']
Predicción: [0.45106542]

Crítica: ['the', 'movie', 'is', '<UNK>']
Predicción: [0.19196399]

